In [30]:
import pandas as pd
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
dfDeliveries = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\IPL_Ball_by_Ball_2008_2022.csv")
dfMatches = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\archive\IPL_Matches_2008_2022.csv")
dfDeliveries = dfDeliveries.merge(dfMatches, on='ID')
dfDeliveries['BowlingTeam'] = np.where(dfDeliveries['BattingTeam']==dfDeliveries['Team1'],
                                       dfDeliveries['Team2'],
                                       dfDeliveries['Team1'])



dfDeliveries['delivery'] = 1
dfDeliveries['Dot_Balls'] = (dfDeliveries['batsman_run'] == 0).astype(int)
dfDeliveries['Singles'] = (dfDeliveries['batsman_run'] == 1).astype(int)
dfDeliveries['Fours'] = (dfDeliveries['batsman_run'] == 4).astype(int)
dfDeliveries['Sixes'] = (dfDeliveries['batsman_run'] == 6).astype(int)


def overClassifier(val):
    if val>=0 and val<6:
        return 'Power Play'
    elif val>=6 and val<15:
        return 'Middle Overs'
    else:
        return 'Death Overs'
dfDeliveries['PhaseOfPlay'] = dfDeliveries['overs'].map(overClassifier)
dfDeliveries[['overs','PhaseOfPlay']].sort_values('overs').drop_duplicates().reset_index(drop=True)
dfDeathOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Death Overs']
dfPowerplayOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Power Play Overs']
dfMiddleOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Middle Overs']

df=dfDeliveries
dfbatting = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\all_bat.csv")
dfbowling = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\all_bowl.csv")
dfvenue= pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\venue.csv")
dfplayervsplayer=pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\Major-CricSim-main\pvp_all.csv")

In [31]:
dfbatting.columns

Index(['batter_id', 'batter', 'Active Ratio_death', 'Striking Ratio_death',
       'True SR_death', 'True Avg_death', 'Active Ratio_middle',
       'Striking Ratio_middle', 'True SR_middle', 'True Avg_middle',
       'Active Ratio_powerplay', 'Striking Ratio_powerplay',
       'True SR_powerplay', 'True Avg_powerplay'],
      dtype='object')

In [32]:
dfbowling.columns

Index(['bowler_id', 'bowler', 'True Economy_death', 'True SR_death',
       'dotpercentage_death', 'Containment_death', 'True Economy_middle',
       'True SR_middle', 'dotpercentage_middle', 'Containment_middle',
       'True Economy_powerplay', 'True SR_powerplay',
       'dotpercentage_powerplay', 'Containment_powerplay'],
      dtype='object')

In [113]:
df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'BowlingTeam', 'delivery', 'Dot_Balls', 'Singles', 'Fours', 'Sixes',
       'PhaseOfPlay'],
      dtype='object')

In [114]:
dfvenue.columns

Index(['venue_name', 'Matches', 'total_run', 'Dot_Balls', 'isWicketDelivery',
       'delivery', 'Fours', 'Sixes', 'Average Score', 'Dot Ball Percentage',
       'Average Wickets Fallen', 'Boundary Frequency'],
      dtype='object')

In [115]:
dfplayervsplayer.columns

Index(['batter', 'bowler', 'dot_percent_death', 'boundary_percent_death',
       'runsperwicket_death', 'runsperball_death', 'dot_percent_middle',
       'boundary_percent_middle', 'runsperwicket_middle', 'runsperball_middle',
       'dot_percent_powerplay', 'boundary_percent_powerplay', 'runsperwicket',
       'runsperball_powerplay'],
      dtype='object')

In [43]:
## VERSION 1
class Player:
    def __init__(self, name, role, batting_avg=None, batting_sr=None, bowling_economy=None, bowling_sr=None, fielding_skill=None):
        self.name = name
        self.role = role
        self.batting_avg = batting_avg
        self.batting_sr = batting_sr
        self.bowling_economy = bowling_economy
        self.bowling_sr = bowling_sr
        self.fielding_skill = fielding_skill
        self.bowled_overs = 0
class Pitch:
    def __init__(self, condition):
        # condition is a string that can be "batsman_friendly", "balanced", or "bowler_friendly"
        self.condition = condition       
class Team:
    def __init__(self, name, players):
        self.name = name
        self.players = players
        self.score = 0
        self.wickets_lost = 0
        self.overs_bowled = 0

def simulate_innings(batting_team, bowling_team, pitch, max_overs=20):
    current_batsman_index = 0
    non_striker_index = 1
    overs = 0
    total_score = 0
    wickets = 0

    while overs < max_overs and wickets < 10:
        bowler = bowling_team.players[overs % len(bowling_team.players)]  # Simple rotation, can be improved
        for ball in range(1, 7):  # Assuming no extras for simplicity
            if wickets >= 10:
                break
            batsman = batting_team.players[current_batsman_index]
            outcome = simulate_ball(batsman, bowler, pitch)
            outcome_to_runs = {"dot": 0, "single": 1, "double": 2, "triple": 3, "four": 4, "six": 6, "extra": 1, "wicket": 0}
            runs_scored = outcome_to_runs.get(outcome, 0)
            total_score += runs_scored

            if outcome == "wicket":
                wickets += 1
                current_batsman_index += 1
                if current_batsman_index == non_striker_index:  # Avoid having the same batsman and non-striker
                    current_batsman_index += 1

            print(f"{batting_team.name} | Over: {overs}.{ball} | {batsman.name} | {bowler.name} | Runs: {runs_scored} | Total: {total_score}/{wickets}")

            if ball == 6:  # Swap the batsmen at the end of the over
                current_batsman_index, non_striker_index = non_striker_index, current_batsman_index

        overs += 1

    batting_team.score = total_score
    batting_team.wickets_lost = wickets
    bowling_team.overs_bowled = overs
    print(f"Innings complete: {batting_team.name} {total_score}/{wickets} after {overs} overs")
def simulate_ball(batsman, bowler, pitch):
    # Adjust outcome probabilities based on player attributes and pitch condition
    base_probability = {
        "dot": 30,
        "single": 40,
        "double": 10,
        "triple": 5,
        "four": 10,
        "six": 4,
        "wicket": 1
    }

    # Adjust for batsman strength
    if batsman.batting_sr > 130:
        base_probability["four"] += 2
        base_probability["six"] += 2
        base_probability["dot"] -= 4

    # Adjust for bowler strength
    if bowler.bowling_economy < 7:
        base_probability["wicket"] += 2
        base_probability["dot"] += 3
        base_probability["four"] -= 2
        base_probability["six"] -= 2

    # Adjust for pitch condition
    if pitch.condition == "batsman_friendly":
        base_probability["four"] += 3
        base_probability["six"] += 2
    elif pitch.condition == "bowler_friendly":
        base_probability["dot"] += 5
        base_probability["wicket"] += 2

    # Determine the outcome of the ball
    outcome = random.choices(list(base_probability.keys()), weights=base_probability.values(), k=1)[0]
    return outcome

# Setup Teams
team1_players = [Player(f"Team1_Player{i}", "batsman", batting_avg=35+i, batting_sr=150+i*5, bowling_economy=2, bowling_sr=10+i) for i in range(1, 12)]
team2_players = [Player(f"Team2_Player{i}", "bowler", batting_avg=25+i, batting_sr=100+i*5, bowling_economy=7.5, bowling_sr=18+i) for i in range(1, 12)]

team1 = Team("Team 1", team1_players)
team2 = Team("Team 2", team2_players)
pitch = Pitch("balanced")

# Simulate Innings
print("First Innings:")
simulate_innings(team1, team2, pitch)

print("\nSecond Innings:")
simulate_innings(team2, team1, pitch, 20)

First Innings:
Team 1 | Over: 0.1 | Team1_Player1 | Team2_Player1 | Runs: 1 | Total: 1/0
Team 1 | Over: 0.2 | Team1_Player1 | Team2_Player1 | Runs: 0 | Total: 1/0
Team 1 | Over: 0.3 | Team1_Player1 | Team2_Player1 | Runs: 1 | Total: 2/0
Team 1 | Over: 0.4 | Team1_Player1 | Team2_Player1 | Runs: 4 | Total: 6/0
Team 1 | Over: 0.5 | Team1_Player1 | Team2_Player1 | Runs: 4 | Total: 10/0
Team 1 | Over: 0.6 | Team1_Player1 | Team2_Player1 | Runs: 0 | Total: 10/0
Team 1 | Over: 1.1 | Team1_Player2 | Team2_Player2 | Runs: 0 | Total: 10/0
Team 1 | Over: 1.2 | Team1_Player2 | Team2_Player2 | Runs: 1 | Total: 11/0
Team 1 | Over: 1.3 | Team1_Player2 | Team2_Player2 | Runs: 3 | Total: 14/0
Team 1 | Over: 1.4 | Team1_Player2 | Team2_Player2 | Runs: 1 | Total: 15/0
Team 1 | Over: 1.5 | Team1_Player2 | Team2_Player2 | Runs: 1 | Total: 16/0
Team 1 | Over: 1.6 | Team1_Player2 | Team2_Player2 | Runs: 1 | Total: 17/0
Team 1 | Over: 2.1 | Team1_Player1 | Team2_Player3 | Runs: 4 | Total: 21/0
Team 1 | Over:

In [38]:
import pandas as pd



class Player:
    def __init__(self, name, player_type= None):
        self.name = name
        self.player_type = player_type
    def __str__(self):
        return f"Player Name: {self.name}, Type: {self.player_type}"

class Batsman(Player):
    def __init__(self, name, **kwargs):
        super().__init__(name, "batsman")
        
        self.active_ratio_death = kwargs.get('Active Ratio_death')
        self.striking_ratio_death = kwargs.get('Striking Ratio_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.true_avg_death = kwargs.get('True Avg_death')
        self.active_ratio_middle = kwargs.get('Active Ratio_middle')
        self.striking_ratio_middle = kwargs.get('Striking Ratio_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.true_avg_middle = kwargs.get('True Avg_middle')
        self.active_ratio_powerplay = kwargs.get('Active Ratio_powerplay')
        self.striking_ratio_powerplay = kwargs.get('Striking Ratio_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.true_avg_powerplay = kwargs.get('True Avg_powerplay')
    def __str__(self):
        return f"Batsman: {self.name}, True Strike Rate (Death): {self.true_sr_death}"



class Bowler(Player):
    def __init__(self, name, **kwargs):
        super().__init__(name, "bowler")
       
        self.true_economy_death = kwargs.get('True Economy_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.dot_percentage_death = kwargs.get('dotpercentage_death')
        self.containment_death = kwargs.get('Containment_death')
        self.true_economy_middle = kwargs.get('True Economy_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.dot_percentage_middle = kwargs.get('dotpercentage_middle')
        self.containment_middle = kwargs.get('Containment_middle')
        self.true_economy_powerplay = kwargs.get('True Economy_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.dot_percentage_powerplay = kwargs.get('dotpercentage_powerplay')
        self.containment_powerplay = kwargs.get('Containment_powerplay')
    def __str__(self):
        return f"Bowler: {self.name}, Containment (Death): {self.containment_death}"
        # def __str__(self):
       #     return f"Bowler: {self.name}, Bowling Economy: {self.true_economy_death}"
class PlayerVsPlayer:
    def __init__(self, batter, bowler, **kwargs):
        self.batter = batter
        self.bowler = bowler
        self.dot_percent_death = kwargs.get('dot_percent_death')
        self.boundary_percent_death = kwargs.get('boundary_percent_death')
        self.runsperwicket_death = kwargs.get('runsperwicket_death')
        self.runsperball_death = kwargs.get('runsperball_death')
        self.dot_percent_middle = kwargs.get('dot_percent_middle')
        self.boundary_percent_middle = kwargs.get('boundary_percent_middle')
        self.runsperwicket_middle = kwargs.get('runsperwicket_middle')
        self.runsperball_middle = kwargs.get('runsperball_middle')
        self.dot_percent_powerplay = kwargs.get('dot_percent_powerplay')
        self.boundary_percent_powerplay = kwargs.get('boundary_percent_powerplay')
        self.runsperwicket_powerplay = kwargs.get('runsperwicket')
        self.runsperball_powerplay = kwargs.get('runsperball_powerplay')


    
class Venue:
    def __init__(self, venue_name, matches, total_run, dot_balls, is_wicket_delivery, delivery, fours, sixes, average_score, dot_ball_percentage, average_wickets_fallen, boundary_frequency):
        self.venue_name = venue_name
        self.matches = matches
        self.total_run = total_run
        self.dot_balls = dot_balls
        self.is_wicket_delivery = is_wicket_delivery
        self.delivery = delivery
        self.fours = fours
        self.sixes = sixes
        self.average_score = average_score
        self.dot_ball_percentage = dot_ball_percentage
        self.average_wickets_fallen = average_wickets_fallen
        self.boundary_frequency = boundary_frequency

def get_player_all_profiles(playing_xi_team1, playing_xi_team2, dfbatting, dfbowling):
    # Initialize empty dicts to hold player profiles
    player_profiles = {}
    
    # Combine both teams into a single list to streamline processing
    all_players = set(playing_xi_team1 + playing_xi_team2)
    
    # Search and Extract Profiles for Both Batting and Bowling
    for player in all_players:
        player_profiles[player] = {
            'batting': None,
            'bowling': None
        }
        
        # Check batting stats
        if player in dfbatting['batter'].values:
            player_profiles[player]['batting'] = dfbatting[dfbatting['batter'] == player].to_dict('records')[0]
        
        # Check bowling stats
        if player in dfbowling['bowler'].values:
            player_profiles[player]['bowling'] = dfbowling[dfbowling['bowler'] == player].to_dict('records')[0]
        
    return player_profiles


def get_venue_features(dfvenue, venue_name):
    # Filter the dataframe for the given venue_name
    venue_data = dfvenue[dfvenue['venue_name'] == venue_name].iloc[0]
    
    # Extract features from the filtered dataframe row
    venue = Venue(
        venue_name=venue_data['venue_name'],
        matches=venue_data['Matches'],
        total_run=venue_data['total_run'],
        dot_balls=venue_data['Dot_Balls'],
        is_wicket_delivery=venue_data['isWicketDelivery'],
        delivery=venue_data['delivery'],
        fours=venue_data['Fours'],
        sixes=venue_data['Sixes'],
        average_score=venue_data['Average Score'],
        dot_ball_percentage=venue_data['Dot Ball Percentage'],
        average_wickets_fallen=venue_data['Average Wickets Fallen'],
        boundary_frequency=venue_data['Boundary Frequency']
    )
    
    return venue


def get_player_vs_player_info(dfplayervsplayer, batter, bowler):
    player_vs_player_info = []

    # Check if the batter and bowler combination exists in the dataframe
    mask = (dfplayervsplayer['batter'] == batter) & (dfplayervsplayer['bowler'] == bowler)
    if mask.any():
        player_vs_player_data = dfplayervsplayer[mask].iloc[0]
        # Create Batsman object
        batsman = Batsman(
            name=player_vs_player_data['batter'],
            # Add any additional attributes specific to Batsman class
        )
        # Create Bowler object
        bowler = Bowler(
            name=player_vs_player_data['bowler'],
            # Add any additional attributes specific to Bowler class
        )
        # Create PlayerVsPlayer object with Batsman and Bowler objects
        player_vs_player_info = PlayerVsPlayer(
            batter=batsman,
            bowler=bowler,
            dot_percent_death=player_vs_player_data.get('dot_percent_death'),
            boundary_percent_death=player_vs_player_data.get('boundary_percent_death'),
            runsperwicket_death=player_vs_player_data.get('runsperwicket_death'),
            runsperball_death=player_vs_player_data.get('runsperball_death'),
            dot_percent_middle=player_vs_player_data.get('dot_percent_middle'),
            boundary_percent_middle=player_vs_player_data.get('boundary_percent_middle'),
            runsperwicket_middle=player_vs_player_data.get('runsperwicket_middle'),
            runsperball_middle=player_vs_player_data.get('runsperball_middle'),
            dot_percent_powerplay=player_vs_player_data.get('dot_percent_powerplay'),
            boundary_percent_powerplay=player_vs_player_data.get('boundary_percent_powerplay'),
            runsperwicket=player_vs_player_data.get('runsperwicket'),
            runsperball_powerplay=player_vs_player_data.get('runsperball_powerplay')
        )
    return [player_vs_player_info] if player_vs_player_info else []



def get_player_vs_player_info_for_teams(dfplayervsplayer, team1, team2):
    player_vs_player_info = []

    # Iterate over each batsman in Team 1 and each bowler in Team 2
    for batter in team1:
        for bowler in team2:
            player_vs_player_info.extend(get_player_vs_player_info(dfplayervsplayer, batter, bowler))

    # Iterate over each batsman in Team 2 and each bowler in Team 1
    for batter in team2:
        for bowler in team1:
            player_vs_player_info.extend(get_player_vs_player_info(dfplayervsplayer, batter, bowler))

    return player_vs_player_info
def combine_player_objects(player_objects):
    combined_list = []
    for player_name, roles in player_objects.items():
        if roles['batting']:
            combined_list.append(roles['batting'])
        if roles['bowling']:
            combined_list.append(roles['bowling'])
    return combined_list
def get_player_objects(playing_xi, dfbatting, dfbowling):
    player_objects = {}

    # Loop through the players in the playing XI
    for player_name in playing_xi:
        player_objects[player_name] = {
            'batting': None,
            'bowling': None
        }

        # Check for batting stats and create a Batsman object if present
        if player_name in dfbatting['batter'].values:
            batting_stats = dfbatting[dfbatting['batter'] == player_name].iloc[0].to_dict()
            player_objects[player_name]['batting'] = Batsman(player_name, **batting_stats)
        
        # Check for bowling stats and create a Bowler object if present
        if player_name in dfbowling['bowler'].values:
            bowling_stats = dfbowling[dfbowling['bowler'] == player_name].iloc[0].to_dict()
            player_objects[player_name]['bowling'] = Bowler(player_name, **bowling_stats)
    
    return player_objects

    



In [60]:
import random
def filter_players_by_type(players, player_type):
    """
    Filter player objects by type (Batsman or Bowler).

    Parameters:
    - players: A list of player objects.
    - player_type: The class type to filter by (Batsman or Bowler).

    Returns:
    - A list of players of the specified type.
    """
    return [player for player in players if isinstance(player, player_type)]


class MatchSimulation:
    def __init__(self, team1, team2, venue):
        self.team1 = team1  # These should now be lists of player objects
        self.team2 = team2  # These should now be lists of player objects
        self.venue = venue
        self.toss_winner = None
        self.toss_choice = None
        self.innings = 1
        self.team_scores = [0, 0]  # Total score for each team
        self.wickets = [0, 0]  # Wickets fallen for each team
        self.overs = [0, 0]  # Overs bowled for each team
        self.current_batsmen = [None, None]  # Current batsmen at the crease
        self.current_bowler = None  # Current bowler
        
    def conduct_toss(self):
        # Simulate a toss
        self.toss_winner = random.choice([self.team1, self.team2])
        self.toss_choice = random.choice(['bat', 'bowl'])
        print(f"{self.toss_winner} won the toss and chose to {self.toss_choice} first.")
        
    def setup_innings(self):
        if self.toss_choice == 'bat' and self.innings == 1 or self.toss_choice == 'bowl' and self.innings == 2:
            batting_team = self.toss_winner
        else:
            batting_team = self.team2 if self.toss_winner == self.team1 else self.team1
        bowling_team = self.team1 if batting_team == self.team2 else self.team2
        
        # Assuming batting_team and bowling_team are already lists of player objects
        # Filter out the batsmen and bowlers
        batting_team_batsmen = filter_players_by_type(batting_team, Batsman)
        bowling_team_bowlers = filter_players_by_type(bowling_team, Bowler)
    
        print(f"Innings {self.innings}: {batting_team} to bat, {bowling_team} to bowl.")
        return batting_team_batsmen, bowling_team_bowlers



    def simulate_innings(self, batting_team, bowling_team, target=None):
        self.current_batsmen = [batting_team[0], batting_team[1]]  # Assuming top 2 are opening batsmen
        team_score = 0
        wickets = 0
        for over in range(1, 21):  # Simulating 20 overs
            self.current_bowler = bowling_team[(over-1) % len(bowling_team)]  # Rotate bowlers
            for ball in range(1, 7):  # Simulating 6 balls per over
                if wickets >= 10:
                    break  # All out
                if self.current_batsmen[0] is None:
                    break  # No more batsmen left
                outcome, runs = self.simulate_ball(self.current_batsmen[0], self.current_bowler)
                team_score += runs
                if outcome == "wicket":
                    wickets += 1
                    # Update current batsman to next in the lineup
                    self.current_batsmen[0] = batting_team[wickets + 1] if wickets + 1 < len(batting_team) else None
                if self.innings == 2:
                    print(f"{over}.{ball} - {self.current_batsmen[0]} faces {self.current_bowler}: {outcome}, Runs: {runs}, Score: {team_score}/{wickets}, Target: {target}")
                    if team_score >= target:
                        print("Chasing team wins!")
                        return team_score
                else:
                    print(f"{over}.{ball} - {self.current_batsmen[0]} faces {self.current_bowler}: {outcome}, Runs: {runs}, Score: {team_score}/{wickets}")
    
        return team_score  # Return the team score after innings completion


    def select_bowler(self, bowling_team_objects):
        # Select a bowler for the over from the list of Bowler objects
        return random.choice(bowling_team_objects)

    def select_batsman(self, batting_team_objects):
        # Select a batsman for the ball from the list of Batsman objects
        return random.choice(batting_team_objects)
    
        # Initialization code as before
        
    # Other methods as before...

    def simulate_ball(self, batsman, bowler):
        # Ensure that 'batsman' is indeed a Batsman object
        if not isinstance(batsman, Batsman):
            print(f"Error: Expected a Batsman object, got {type(batsman)} instead.")
            return None, 0  # Indicates an error in object type
    
        if self.overs[self.innings-1] < 6:
            phase = 'powerplay'
        elif self.overs[self.innings-1] < 16:
            phase = 'middle'
        else:
            phase = 'death'
    
        outcome_probabilities = self.calculate_outcome_probabilities(batsman, bowler, phase)
    
        outcomes = ['dot', 'single', 'double', 'triple', 'four', 'six', 'wicket']
        outcome = random.choices(outcomes, weights=outcome_probabilities, k=1)[0]
    
        # Map the outcome to runs scored
        runs_scored = 0 if outcome in ['dot', 'wicket'] else int(outcome) if outcome in ['single', 'double', 'triple', 'four', 'six'] else 0
    
        print(f"{batsman} faces {bowler}: Outcome - {outcome}, Runs scored: {runs_scored}")
    
        # Return the outcome and runs scored for further processing
        return outcome, runs_scored


    def update_match_state(self, outcome, batsman, bowler):
        innings_index = self.innings - 1  # Adjust for 0-based indexing
        
        if outcome == 'dot':
            pass  # No runs scored
        elif outcome == 'single':
            self.team_scores[innings_index] += 1
        elif outcome == 'double':
            self.team_scores[innings_index] += 2
        elif outcome == 'triple':
            self.team_scores[innings_index] += 3
        elif outcome == 'four':
            self.team_scores[innings_index] += 4
        elif outcome == 'six':
            self.team_scores[innings_index] += 6
        elif outcome == 'wicket':
            self.wickets[innings_index] += 1
            # Handle batsman change if necessary
            
        # Update the number of balls bowled in the current over
        balls_in_over = (self.overs[innings_index] * 6) % 6 + 1
        if balls_in_over == 6:  # An over has been completed
            self.overs[innings_index] += 1
        else:
            self.overs[innings_index] += 1 / 6  # Increment by one-sixth for each ball
        
        print(f"Updated score: {self.team_scores[innings_index]}/{self.wickets[innings_index]} after {self.overs[innings_index]:.1f} overs")


    
    def calculate_outcome_probabilities(self, batsman, bowler, phase):
        # Base probabilities and stats
        base_probabilities = {'dot': 30, 'single': 20, 'double': 10, 'triple': 5, 'four': 15, 'six': 10, 'wicket': 10}
        baseSR = 100  # Simplistic base strike rate
        baseAVG = 30  # Simplistic base average

        # Define outcomes here to ensure it's available within this method
        outcomes = ['dot', 'single', 'double', 'triple', 'four', 'six', 'wicket']

        # Determine phase-specific stats for the batsman
        if phase == 'powerplay':
            trueSR = batsman.true_sr_powerplay
            trueAVG = batsman.true_avg_powerplay
            active_ratio = batsman.active_ratio_powerplay
        elif phase == 'middle':
            trueSR = batsman.true_sr_middle
            trueAVG = batsman.true_avg_middle
            active_ratio = batsman.active_ratio_middle
        else:  # death
            trueSR = batsman.true_sr_death
            trueAVG = batsman.true_avg_death
            active_ratio = batsman.active_ratio_death

        # Adjustments as before...
        
        # Normalize probabilities to ensure they sum to 100
        total_probability = sum(base_probabilities.values())
        normalized_probabilities = [base_probabilities[outcome] / total_probability * 100 for outcome in outcomes]

        return normalized_probabilities
    #def filter_players_by_type(players, player_type):
        #return [player for player in players if isinstance(player, player_type)]


    # Update match state as before...

# Remember, this class now expects team1 and team2 to be lists of Batsman and Bowler objects.
# Make sure to transform your team lists from names to objects before using this class.




In [61]:
team1_names = ["A Symonds", "AC Gilchrist","SR Watson","AD Russell","BA Stokes","CH Gayle","CH Morris","JC Archer","KA Pollard","KP Pietersen","Shakib Al Hasan"] 
team2_names = ["SR Tendulkar", "Z Khan","RG Sharma","V Kohli","MS Dhoni","KH Pandya","HH Pandya","JJ Bumrah","SK Raina","YK Pathan","YS Chahal"]
player_profile = get_player_all_profiles(team1, team2, dfbatting, dfbowling)
playervsplayer_profile = get_player_vs_player_info_for_teams(dfplayervsplayer, team1, team2)
player_stats = player_profile
batsmen = []
bowlers = []

for player_name, stats in player_stats.items():
    if 'batting' in stats and stats['batting'] is not None:
        batsmen.append(Batsman(player_name, **stats['batting']))
    if 'bowling' in stats and stats['bowling'] is not None:
        bowlers.append(Bowler(player_name, **stats['bowling']))
venue_name = "Brabourne Stadium"
venue_features = get_venue_features(dfvenue, venue_name)

# Assuming 'batsmen' and 'bowlers' lists are already populated with Batsman and Bowler objects
# Create a mapping from player names to their respective objects
player_objects_mapping = {}
for player in batsmen + bowlers:
    player_objects_mapping[player.name] = player

player_objects_team1 = get_player_objects(team1_names, dfbatting, dfbowling)
player_objects_team2 = get_player_objects(team2_names, dfbatting, dfbowling)
team1_objects = combine_player_objects(player_objects_team1)
team2_objects = combine_player_objects(player_objects_team2)

match_simulation = MatchSimulation(team1_objects, team2_objects, venue_name)
# # Proceed with the toss and innings simulation as before
# match_simulation.conduct_toss()
# batting_team, bowling_team = match_simulation.setup_innings()
# # Assuming setup_innings now returns lists of objects based on the toss
# match_simulation.simulate_innings(batting_team, bowling_team)

# # Switch innings
# match_simulation.innings = 2
# batting_team, bowling_team = match_simulation.setup_innings()
# match_simulation.simulate_innings(batting_team, bowling_team)
# Example flow
first_innings_score = match_simulation.simulate_innings(team1, team2)
target = first_innings_score + 1
print(f"Target for the second team: {target}")
match_simulation.simulate_innings(team2, team1, target=target)


Error: Expected a Batsman object, got <class 'str'> instead.
1.1 - A Symonds faces SR Tendulkar: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
1.2 - A Symonds faces SR Tendulkar: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
1.3 - A Symonds faces SR Tendulkar: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
1.4 - A Symonds faces SR Tendulkar: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
1.5 - A Symonds faces SR Tendulkar: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
1.6 - A Symonds faces SR Tendulkar: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
2.1 - A Symonds faces Z Khan: None, Runs: 0, Score: 0/0
Error: Expected a Batsman object, got <class 'str'> instead.
2.2 - A Symonds faces Z Khan: None, Runs: 0, Score: 0/0
Error: Expected a Batsman ob

0

In [132]:
def get_player_vs_player_info(dfplayervsplayer, batter, bowler):
    player_vs_player_info = []

    # Check if the batter and bowler combination exists in the dataframe
    mask = (dfplayervsplayer['batter'] == batter) & (dfplayervsplayer['bowler'] == bowler)
    if mask.any():
        player_vs_player_data = dfplayervsplayer[mask].iloc[0]
        # Create Batsman object
        batsman = Batsman(
            name=player_vs_player_data['batter'],
            # Add any additional attributes specific to Batsman class
        )
        # Create Bowler object
        bowler = Bowler(
            name=player_vs_player_data['bowler'],
            # Add any additional attributes specific to Bowler class
        )
        # Create PlayerVsPlayer object with Batsman and Bowler objects
        player_vs_player_info = PlayerVsPlayer(
            batter=batsman,
            bowler=bowler,
            dot_percent_death=player_vs_player_data.get('dot_percent_death'),
            boundary_percent_death=player_vs_player_data.get('boundary_percent_death'),
            runsperwicket_death=player_vs_player_data.get('runsperwicket_death'),
            runsperball_death=player_vs_player_data.get('runsperball_death'),
            dot_percent_middle=player_vs_player_data.get('dot_percent_middle'),
            boundary_percent_middle=player_vs_player_data.get('boundary_percent_middle'),
            runsperwicket_middle=player_vs_player_data.get('runsperwicket_middle'),
            runsperball_middle=player_vs_player_data.get('runsperball_middle'),
            dot_percent_powerplay=player_vs_player_data.get('dot_percent_powerplay'),
            boundary_percent_powerplay=player_vs_player_data.get('boundary_percent_powerplay'),
            runsperwicket=player_vs_player_data.get('runsperwicket'),
            runsperball_powerplay=player_vs_player_data.get('runsperball_powerplay')
        )
    return [player_vs_player_info] if player_vs_player_info else []



def get_player_vs_player_info_for_teams(dfplayervsplayer, team1, team2):
    player_vs_player_info = []

    # Iterate over each batsman in Team 1 and each bowler in Team 2
    for batter in team1:
        for bowler in team2:
            player_vs_player_info.extend(get_player_vs_player_info(dfplayervsplayer, batter, bowler))

    # Iterate over each batsman in Team 2 and each bowler in Team 1
    for batter in team2:
        for bowler in team1:
            player_vs_player_info.extend(get_player_vs_player_info(dfplayervsplayer, batter, bowler))

    return player_vs_player_info

    
playervsplayer_profile = get_player_vs_player_info_for_teams(dfplayervsplayer, team1, team2)

# Iterate over each PlayerVsPlayer object and print the information
for pvp in playervsplayer_profile:
    print(f"Batter: {pvp.batter.name}, Bowler: {pvp.bowler.name}")
    print(f"Dot Percent (Death): {pvp.dot_percent_death}")
    print(f"Boundary Percent (Death): {pvp.boundary_percent_death}")
    print(f"Runs Per Wicket (Death): {pvp.runsperwicket_death}")
    print(f"Runs Per Ball (Death): {pvp.runsperball_death}")
    print(f"Dot Percent (Middle): {pvp.dot_percent_middle}")
    print(f"Boundary Percent (Middle): {pvp.boundary_percent_middle}")
    print(f"Runs Per Wicket (Middle): {pvp.runsperwicket_middle}")
    print(f"Runs Per Ball (Middle): {pvp.runsperball_middle}")
    print(f"Dot Percent (Powerplay): {pvp.dot_percent_powerplay}")
    print(f"Boundary Percent (Powerplay): {pvp.boundary_percent_powerplay}")
    print(f"Runs Per Wicket (Powerplay): {pvp.runsperwicket_powerplay}")
    print(f"Runs Per Ball (Powerplay): {pvp.runsperball_powerplay}")
    print()


Batter: A Symonds, Bowler: Z Khan
Dot Percent (Death): 0.0
Boundary Percent (Death): 0.0
Runs Per Wicket (Death): 3.0
Runs Per Ball (Death): 1.0
Dot Percent (Middle): 0.0
Boundary Percent (Middle): 0.0
Runs Per Wicket (Middle): 3.0
Runs Per Ball (Middle): 1.0
Dot Percent (Powerplay): 0.0
Boundary Percent (Powerplay): 0.0
Runs Per Wicket (Powerplay): 3.0
Runs Per Ball (Powerplay): 1.0

Batter: A Symonds, Bowler: V Kohli
Dot Percent (Death): 0.2857142857142857
Boundary Percent (Death): 0.1428571428571428
Runs Per Wicket (Death): 11.0
Runs Per Ball (Death): 1.5714285714285714
Dot Percent (Middle): 0.2857142857142857
Boundary Percent (Middle): 0.1428571428571428
Runs Per Wicket (Middle): 11.0
Runs Per Ball (Middle): 1.5714285714285714
Dot Percent (Powerplay): 0.2857142857142857
Boundary Percent (Powerplay): 0.1428571428571428
Runs Per Wicket (Powerplay): 11.0
Runs Per Ball (Powerplay): 1.5714285714285714

Batter: A Symonds, Bowler: SK Raina
Dot Percent (Death): 0.375
Boundary Percent (Deat

In [127]:


class PlayerVsPlayer:
    def __init__(self, batter, bowler, **kwargs):
        self.batter = batter
        self.bowler = bowler
        self.dot_percent_death = kwargs.get('dot_percent_death')
        self.boundary_percent_death = kwargs.get('boundary_percent_death')
        self.runsperwicket_death = kwargs.get('runsperwicket_death')
        self.runsperball_death = kwargs.get('runsperball_death')
        self.dot_percent_middle = kwargs.get('dot_percent_middle')
        self.boundary_percent_middle = kwargs.get('boundary_percent_middle')
        self.runsperwicket_middle = kwargs.get('runsperwicket_middle')
        self.runsperball_middle = kwargs.get('runsperball_middle')
        self.dot_percent_powerplay = kwargs.get('dot_percent_powerplay')
        self.boundary_percent_powerplay = kwargs.get('boundary_percent_powerplay')
        self.runsperwicket_powerplay = kwargs.get('runsperwicket')
        self.runsperball_powerplay = kwargs.get('runsperball_powerplay')

def compute_player_vs_player_stats(player_profiles, dfplayervsplayer):
    # Iterate through player profiles
    for player, profile in player_profiles.items():
        print(f"Player: {player}")
        
        # Check if both batting and bowling profiles exist
        if profile['batting'] and profile['bowling']:
            batter_name = profile['batting']['batter']
            bowler_name = profile['bowling']['bowler']
            
            # Get player-vs-player info
            player_vs_player_info = get_player_vs_player_info(dfplayervsplayer, batter_name, bowler_name)
            
            # Add player-vs-player info to player profile if available
            if player_vs_player_info:
                print("Player vs Player Stats:")
                print(f"Batter: {player_vs_player_info.batter.name}")
                print(f"Bowler: {player_vs_player_info.bowler.name}")
                print(f"Dot Percentage (Death): {player_vs_player_info.dot_percent_death}")
                print(f"Boundary Percentage (Death): {player_vs_player_info.boundary_percent_death}")
                print(f"Runs per Wicket (Death): {player_vs_player_info.runsperwicket_death}")
                print(f"Runs per Ball (Death): {player_vs_player_info.runsperball_death}")
                # Print other relevant attributes as needed
            else:
                print("No player-vs-player stats available.")
        else:
            print("No batting or bowling profile available.")

# Now you need to call compute_player_vs_player_stats function passing player_profile and dfplayervsplayer
compute_player_vs_player_stats(player_profile, dfplayervsplayer)


# Now you need to call compute_player_vs_player_stats function passing player_profile and dfplayervsplayer
#compute_player_vs_player_stats(player_profile, dfplayervsplayer)

Player: JC Archer
No player-vs-player stats available.
Player: Shakib Al Hasan
No player-vs-player stats available.
Player: YK Pathan
No player-vs-player stats available.
Player: KA Pollard
No player-vs-player stats available.
Player: AC Gilchrist
No batting or bowling profile available.
Player: JJ Bumrah
No batting or bowling profile available.
Player: CH Morris
No player-vs-player stats available.
Player: Z Khan
No player-vs-player stats available.
Player: CH Gayle
No player-vs-player stats available.
Player: BA Stokes
No player-vs-player stats available.
Player: V Kohli
No batting or bowling profile available.
Player: SK Raina
No player-vs-player stats available.
Player: SR Watson
No player-vs-player stats available.
Player: A Symonds
No player-vs-player stats available.
Player: SR Tendulkar
No batting or bowling profile available.
Player: KH Pandya
No player-vs-player stats available.
Player: MS Dhoni
No batting or bowling profile available.
Player: HH Pandya
No player-vs-player st

In [124]:
class PlayerVsPlayer:
    def __init__(self, batter, bowler, **kwargs):
        self.batter = batter
        self.bowler = bowler
        self.dot_percent_death = kwargs.get('dot_percent_death')
        self.boundary_percent_death = kwargs.get('boundary_percent_death')
        self.runsperwicket_death = kwargs.get('runsperwicket_death')
        self.runsperball_death = kwargs.get('runsperball_death')
        self.dot_percent_middle = kwargs.get('dot_percent_middle')
        self.boundary_percent_middle = kwargs.get('boundary_percent_middle')
        self.runsperwicket_middle = kwargs.get('runsperwicket_middle')
        self.runsperball_middle = kwargs.get('runsperball_middle')
        self.dot_percent_powerplay = kwargs.get('dot_percent_powerplay')
        self.boundary_percent_powerplay = kwargs.get('boundary_percent_powerplay')
        self.runsperwicket_powerplay = kwargs.get('runsperwicket')
        self.runsperball_powerplay = kwargs.get('runsperball_powerplay')

def get_player_vs_player_info(dfplayervsplayer, batter, bowler):
    player_vs_player_info = None
    # Check if the batter and bowler combination exists in the dataframe
    mask = (dfplayervsplayer['batter'] == batter) & (dfplayervsplayer['bowler'] == bowler)
    if mask.any():
        player_vs_player_data = dfplayervsplayer[mask].iloc[0]
        # Create Batsman object
        batsman = Batsman(
            name=player_vs_player_data['batter'],
            # Add any additional attributes specific to Batsman class
        )
        # Create Bowler object
        bowler = Bowler(
            name=player_vs_player_data['bowler'],
            # Add any additional attributes specific to Bowler class
        )
        # Create PlayerVsPlayer object with Batsman and Bowler objects
        player_vs_player_info = PlayerVsPlayer(
            batter=batsman,
            bowler=bowler,
            dot_percent_death=player_vs_player_data['dot_percent_death'],
            boundary_percent_death=player_vs_player_data['boundary_percent_death'],
            runsperwicket_death=player_vs_player_data['runsperwicket_death'],
            runsperball_death=player_vs_player_data['runsperball_death'],
            dot_percent_middle=player_vs_player_data['dot_percent_middle'],
            boundary_percent_middle=player_vs_player_data['boundary_percent_middle'],
            runsperwicket_middle=player_vs_player_data['runsperwicket_middle'],
            runsperball_middle=player_vs_player_data['runsperball_middle'],
            dot_percent_powerplay=player_vs_player_data['dot_percent_powerplay'],
            boundary_percent_powerplay=player_vs_player_data['boundary_percent_powerplay'],
            runsperwicket=player_vs_player_data['runsperwicket'],
            runsperball_powerplay=player_vs_player_data['runsperball_powerplay']
        )
    return player_vs_player_info

# Assuming dfplayervsplayer is your DataFrame containing player-vs-player information

# Retrieve player-vs-player information for "RG Sharma" as a batter and bowler
player_vs_player_batter = get_player_vs_player_info(dfplayervsplayer, "RG Sharma", "Z Khan")
player_vs_player_bowler = get_player_vs_player_info(dfplayervsplayer, "RG Sharma", "CH Morris")

# Print the player-vs-player information if available
if player_vs_player_batter is not None:
    print("Player vs Player as Batter:")
    print(f"Batter: {player_vs_player_batter.batter}, Bowler: {player_vs_player_batter.bowler}")
    print(f"Dot Percentage (Death): {player_vs_player_batter.dot_percent_death}")
    print(f"Boundary Percentage (Death): {player_vs_player_batter.boundary_percent_death}")
    print(f"Runs per Wicket (Death): {player_vs_player_batter.runsperwicket_death}")
    print(f"Runs per Ball (Death): {player_vs_player_batter.runsperball_death}")
    # Print other relevant attributes as needed

if player_vs_player_bowler is not None:
    print("\nPlayer vs Player as Bowler:")
    print(f"Batter: {player_vs_player_bowler.batter}, Bowler: {player_vs_player_bowler.bowler}")
    print(f"Dot Percentage (Death): {player_vs_player_bowler.dot_percent_death}")
    print(f"Boundary Percentage (Death): {player_vs_player_bowler.boundary_percent_death}")
    print(f"Runs per Wicket (Death): {player_vs_player_bowler.runsperwicket_death}")
    print(f"Runs per Ball (Death): {player_vs_player_bowler.runsperball_death}")
    # Print other relevant attributes as needed


Player vs Player as Batter:
Batter: Batsman: RG Sharma, True Strike Rate (Death): None, Bowler: Bowler: Z Khan, Containment (Death): None
Dot Percentage (Death): 0.3939393939393939
Boundary Percentage (Death): 0.0909090909090909
Runs per Wicket (Death): 14.5
Runs per Ball (Death): 0.8787878787878788

Player vs Player as Bowler:
Batter: Batsman: RG Sharma, True Strike Rate (Death): None, Bowler: Bowler: CH Morris, Containment (Death): None
Dot Percentage (Death): 0.4193548387096774
Boundary Percentage (Death): 0.1290322580645161
Runs per Wicket (Death): 12.0
Runs per Ball (Death): 1.1612903225806452


In [104]:

# Example usage
venue_name = "Brabourne Stadium"
venue_features = get_venue_features(dfvenue, venue_name)
print(venue_features.venue_name, venue_features.matches, ...)


Brabourne Stadium 27 Ellipsis


In [90]:
def print_dict(d, indent=0):
    for key, value in d.items():
        print('    ' * indent + str(key) + ':')
        if isinstance(value, dict):
            print_dict(value, indent+1)
        else:
            print('    ' * (indent+1) + str(value))
print("Player Profiles:")
print_dict(player_profile)

Player Profiles:
JC Archer:
    batting:
        batter:
            JC Archer
        batter_id:
            427
        Active Ratio_death:
            0.704761905
        Striking Ratio_death:
            0.283783784
        True SR_death:
            7.380952381
        True Avg_death:
            4.956349206
        Active Ratio_middle:
            0.5
        Striking Ratio_middle:
            0.25
        True SR_middle:
            12.18181818
        True Avg_middle:
            -11.0952381
        Active Ratio_powerplay:
            0.0
        Striking Ratio_powerplay:
            0.0
        True SR_powerplay:
            -125.0
        True Avg_powerplay:
            -45.0
    bowling:
        bowler:
            JC Archer
        bowler_id:
            380
        True Economy_death:
            2.0633043265106004
        True SR_death:
            -2.5162790697674406
        dotpercentage_death:
            0.2957317073170731
        Containment_death:
            1.5396

In [99]:
class Player:
    def __init__(self, name, player_type= None):
        self.name = name
        self.player_type = player_type
    def __str__(self):
        return f"Player Name: {self.name}, Type: {self.player_type}"

class Batsman(Player):
    def __init__(self, name, **kwargs):
        super().__init__(name, "batsman")
        
        self.active_ratio_death = kwargs.get('Active Ratio_death')
        self.striking_ratio_death = kwargs.get('Striking Ratio_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.true_avg_death = kwargs.get('True Avg_death')
        self.active_ratio_middle = kwargs.get('Active Ratio_middle')
        self.striking_ratio_middle = kwargs.get('Striking Ratio_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.true_avg_middle = kwargs.get('True Avg_middle')
        self.active_ratio_powerplay = kwargs.get('Active Ratio_powerplay')
        self.striking_ratio_powerplay = kwargs.get('Striking Ratio_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.true_avg_powerplay = kwargs.get('True Avg_powerplay')
    def __str__(self):
        return f"Batsman: {self.name}, True Strike Rate (Death): {self.true_sr_death}"



class Bowler(Player):
    def __init__(self, name, **kwargs):
        super().__init__(name, "bowler")
       
        self.true_economy_death = kwargs.get('True Economy_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.dot_percentage_death = kwargs.get('dotpercentage_death')
        self.containment_death = kwargs.get('Containment_death')
        self.true_economy_middle = kwargs.get('True Economy_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.dot_percentage_middle = kwargs.get('dotpercentage_middle')
        self.containment_middle = kwargs.get('Containment_middle')
        self.true_economy_powerplay = kwargs.get('True Economy_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.dot_percentage_powerplay = kwargs.get('dotpercentage_powerplay')
        self.containment_powerplay = kwargs.get('Containment_powerplay')
    def __str__(self):
        return f"Bowler: {self.name}, Containment (Death): {self.containment_death}"
        # def __str__(self):
        #     return f"Bowler: {self.name}, Bowling Economy: {self.true_economy_death}"
player_stats = player_profile
batsmen = []
bowlers = []

for player_name, stats in player_stats.items():
    if 'batting' in stats and stats['batting'] is not None:
        batsmen.append(Batsman(player_name, **stats['batting']))
    if 'bowling' in stats and stats['bowling'] is not None:
        bowlers.append(Bowler(player_name, **stats['bowling']))

print("Batsmen Profiles:")
for batsman in batsmen:
    print(batsman)

print("\nBowlers Profiles:")
for bowler in bowlers:
    print(bowler)

Batsmen Profiles:
Batsman: JC Archer, True Strike Rate (Death): 7.380952381
Batsman: Shakib Al Hasan, True Strike Rate (Death): -0.616716464
Batsman: YK Pathan, True Strike Rate (Death): 11.13332487
Batsman: KA Pollard, True Strike Rate (Death): 8.475450473
Batsman: AC Gilchrist, True Strike Rate (Death): 77.15355805
Batsman: CH Morris, True Strike Rate (Death): 16.89132218
Batsman: Z Khan, True Strike Rate (Death): -33.99986643
Batsman: CH Gayle, True Strike Rate (Death): 31.36910316
Batsman: BA Stokes, True Strike Rate (Death): 13.25056584
Batsman: V Kohli, True Strike Rate (Death): 13.56631114
Batsman: SK Raina, True Strike Rate (Death): 8.912880656
Batsman: SR Watson, True Strike Rate (Death): 19.21434079
Batsman: A Symonds, True Strike Rate (Death): 7.991875827
Batsman: SR Tendulkar, True Strike Rate (Death): 3.236121154
Batsman: KH Pandya, True Strike Rate (Death): 3.831478537
Batsman: MS Dhoni, True Strike Rate (Death): 10.11227236
Batsman: HH Pandya, True Strike Rate (Death): 1

In [71]:
class Player:
    def __init__(self, name, player_type):
        self.name = name
        self.player_type = player_type
    def __str__(self):
        return f"Player Name: {self.name}, Type: {self.player_type}"

class Batsman(Player):
    def __init__(self, name, **kwargs):
        super().__init__(name, "batsman")
        self.batting_avg = kwargs.get('True Avg')
        self.batting_sr = kwargs.get('Striking Ratio')
        self.active_ratio_death = kwargs.get('Active Ratio_death')
        self.striking_ratio_death = kwargs.get('Striking Ratio_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.true_avg_death = kwargs.get('True Avg_death')
        self.active_ratio_middle = kwargs.get('Active Ratio_middle')
        self.striking_ratio_middle = kwargs.get('Striking Ratio_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.true_avg_middle = kwargs.get('True Avg_middle')
        self.active_ratio_powerplay = kwargs.get('Active Ratio_powerplay')
        self.striking_ratio_powerplay = kwargs.get('Striking Ratio_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.true_avg = kwargs.get('True Avg')

class Bowler(Player):
    def __init__(self, name, **kwargs):
        super().__init__(name, "bowler")
        self.bowling_economy = kwargs.get('True Economy')
        self.bowling_sr = kwargs.get('True SR')
        self.true_economy_death = kwargs.get('True Economy_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.dot_percentage_death = kwargs.get('dotpercentage_death')
        self.containment_death = kwargs.get('Containment_death')
        self.true_economy_middle = kwargs.get('True Economy_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.dot_percentage_middle = kwargs.get('dotpercentage_middle')
        self.containment_middle = kwargs.get('Containment_middle')
        self.true_economy_powerplay = kwargs.get('True Economy_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.dot_percentage_powerplay = kwargs.get('dotpercentage_powerplay')
        self.containment_powerplay = kwargs.get('Containment_powerplay')

# Now, let's create the objects
player_objects = {}

for player, profiles in player_profile.items():
    if profiles['batting'] is not None:
        # Remove keys that don't match constructor parameters
        batting_info = {k.replace('batter_', ''): v for k, v in profiles['batting'].items() if k != 'batter' and k != 'batter_id'}
        player_objects[player] = Batsman(player, **batting_info)
        
    if profiles['bowling'] is not None:
        # Remove keys that don't match constructor parameters
        bowling_info = {k.replace('bowler_', ''): v for k, v in profiles['bowling'].items() if k != 'bowler' and k != 'bowler_id'}
        # Check if player object already exists, update it if so
        if player in player_objects:
            player_objects[player].__dict__.update(Bowler(player, **bowling_info).__dict__)
        else:
            player_objects[player] = Bowler(player, **bowling_info)

# player_objects now contains all the player objects with their respective profiles
for player_name, player_object in player_objects.items():
    print(player_object)


Player Name: JC Archer, Type: bowler
Player Name: Shakib Al Hasan, Type: bowler
Player Name: YK Pathan, Type: bowler
Player Name: KA Pollard, Type: bowler
Player Name: AC Gilchrist, Type: batsman
Player Name: JJ Bumrah, Type: bowler
Player Name: CH Morris, Type: bowler
Player Name: Z Khan, Type: bowler
Player Name: CH Gayle, Type: bowler
Player Name: BA Stokes, Type: bowler
Player Name: V Kohli, Type: batsman
Player Name: SK Raina, Type: bowler
Player Name: SR Watson, Type: bowler
Player Name: A Symonds, Type: bowler
Player Name: SR Tendulkar, Type: batsman
Player Name: KH Pandya, Type: bowler
Player Name: MS Dhoni, Type: batsman
Player Name: HH Pandya, Type: bowler
Player Name: KP Pietersen, Type: bowler
Player Name: YS Chahal, Type: bowler
Player Name: RG Sharma, Type: bowler
Player Name: AD Russell, Type: bowler


In [72]:
class Player:
    def __init__(self, name):
        self.name = name
        self.batsman = None  # This will hold a Batsman object
        self.bowler = None  # This will hold a Bowler object

    def set_batsman(self, **kwargs):
        self.batsman = Batsman(**kwargs)
        
    def set_bowler(self, **kwargs):
        self.bowler = Bowler(**kwargs)

    def __str__(self):
        player_type = 'Player'
        if self.batsman and self.bowler:
            player_type = 'All-Rounder'
        elif self.batsman:
            player_type = 'Batsman'
        elif self.bowler:
            player_type = 'Bowler'
        return f"Name: {self.name}, Type: {player_type}, Batting: {self.batsman}, Bowling: {self.bowler}"

class Batsman:
    def __init__(self, **kwargs):
        super().__init__(name, "batsman")
        self.batting_avg = kwargs.get('True Avg')
        self.batting_sr = kwargs.get('Striking Ratio')
        self.active_ratio_death = kwargs.get('Active Ratio_death')
        self.striking_ratio_death = kwargs.get('Striking Ratio_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.true_avg_death = kwargs.get('True Avg_death')
        self.active_ratio_middle = kwargs.get('Active Ratio_middle')
        self.striking_ratio_middle = kwargs.get('Striking Ratio_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.true_avg_middle = kwargs.get('True Avg_middle')
        self.active_ratio_powerplay = kwargs.get('Active Ratio_powerplay')
        self.striking_ratio_powerplay = kwargs.get('Striking Ratio_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.true_avg = kwargs.get('True Avg')
    def __str__(self):
        return f"Avg: {self.batting_avg}"  # Extend this to include other attributes

class Bowler:
    def __init__(self, **kwargs):
        super().__init__(name, "bowler")
        self.bowling_economy = kwargs.get('True Economy')
        self.bowling_sr = kwargs.get('True SR')
        self.true_economy_death = kwargs.get('True Economy_death')
        self.true_sr_death = kwargs.get('True SR_death')
        self.dot_percentage_death = kwargs.get('dotpercentage_death')
        self.containment_death = kwargs.get('Containment_death')
        self.true_economy_middle = kwargs.get('True Economy_middle')
        self.true_sr_middle = kwargs.get('True SR_middle')
        self.dot_percentage_middle = kwargs.get('dotpercentage_middle')
        self.containment_middle = kwargs.get('Containment_middle')
        self.true_economy_powerplay = kwargs.get('True Economy_powerplay')
        self.true_sr_powerplay = kwargs.get('True SR_powerplay')
        self.dot_percentage_powerplay = kwargs.get('dotpercentage_powerplay')
        self.containment_powerplay = kwargs.get('Containment_powerplay')
    def __str__(self):
        return f"Economy: {self.bowling_economy}"  # Extend this to include other attributes

# Assuming player_profile is your input dictionary
player_objects = {}

for player, profiles in player_profile.items():
    player_obj = Player(player)

    if profiles['batting']:
        batting_info = {k: v for k, v in profiles['batting'].items() if k != 'batter' and k != 'batter_id'}
        player_obj.set_batsman(**batting_info)

    if profiles['bowling']:
        bowling_info = {k: v for k, v in profiles['bowling'].items() if k != 'bowler' and k != 'bowler_id'}
        player_obj.set_bowler(**bowling_info)
    
    player_objects[player] = player_obj

# Now, printing all player objects with their details
for player_name, player_object in player_objects.items():
    print(player_object)


NameError: name 'name' is not defined

In [80]:
import pandas as pd

# Sample dataframes (assuming dfbatting and dfbowling are similar to these)
# dfbatting = pd.DataFrame(...)
# dfbowling = pd.DataFrame(...)

def get_player_all_profiles(playing_xi_team1, playing_xi_team2, dfbatting, dfbowling):
    # Initialize empty dicts to hold player profiles
    player_profiles = {}
    
    # Combine both teams into a single list to streamline processing
    all_players = set(playing_xi_team1 + playing_xi_team2)
    
    # Search and Extract Profiles for Both Batting and Bowling
    for player in all_players:
        player_profiles[player] = {
            'batting': None,
            'bowling': None
        }
        
        # Check batting stats
        if player in dfbatting['batter'].values:
            player_profiles[player]['batting'] = dfbatting[dfbatting['batter'] == player].to_dict('records')[0]
        
        # Check bowling stats
        if player in dfbowling['bowler'].values:
            player_profiles[player]['bowling'] = dfbowling[dfbowling['bowler'] == player].to_dict('records')[0]
        
    return player_profiles

# Example usage
team1 = ["A Symonds", "AC Gilchrist","SR Watson","AD Russell","BA Stokes","CH Gayle","CH Morris","JC Archer","KA Pollard","KP Pietersen","Shakib Al Hasan"] 
team2 = ["SR Tendulkar", "Z Khan","RG Sharma","V Kohli","MS Dhoni","KH Pandya","HH Pandya","JJ Bumrah","SK Raina","YK Pathan","YS Chahal"]
player_profile = get_player_all_profiles(team1, team2, dfbatting, dfbowling)

print("Player Profiles:", player_profile)


class Player:
    def __init__(self, name):
        self.name = name
        self.player_type = "Player"
        # Initialize attributes dynamically as needed

    def set_batsman(self, **kwargs):
        self.player_type = "Batsman" if self.player_type == "Player" else "Allrounder"
        batting_features = ['Active Ratio_death', 'Striking Ratio_death', 'True SR_death', 'True Avg_death',
                            'Active Ratio_middle', 'Striking Ratio_middle', 'True SR_middle', 'True Avg_middle',
                            'Active Ratio', 'Striking Ratio', 'True SR_powerplay', 'True Avg']
        for feature in batting_features:
            setattr(self, feature.lower(), kwargs.get(feature))

    def set_bowler(self, **kwargs):
        self.player_type = "Bowler" if self.player_type == "Player" else "Allrounder"
        bowling_features = ['True Economy_death', 'True SR_death', 'dotpercentage_death', 'Containment_death',
                            'True Economy_middle', 'True SR_middle', 'dotpercentage_middle', 'Containment_middle',
                            'True Economy_powerplay', 'True SR_powerplay', 'dotpercentage_powerplay', 'Containment_powerplay']
        for feature in bowling_features:
            setattr(self, feature.lower(), kwargs.get(feature))
            
    def __str__(self):
        player_info = f"Player Name: {self.name}, Type: {self.player_type}\n"
        batting_info = ""

        # Print batting stats for Batsman and Allrounder types only
        if self.player_type in ["Batsman", "Allrounder"]:
            batting_info = "\n".join([f"{attr.replace('_', ' ').title()}: {getattr(self, attr)}" 
                                      for attr in self.__dict__ 
                                      if attr.startswith("Active Ratio") or attr.startswith("Striking Ratio") 
                                      or attr.startswith("True SR") or attr.startswith("True Avg")])
        
        # If there's batting info, prepend the section title
        batting_info = f"Batting Stats:\n{batting_info}" if batting_info else ""

        return "\n".join([player_info, batting_info]).strip()

# Assuming player_profile is defined as before
player_objects = {}

for player, profiles in player_profile.items():
    player_obj = Player(player)
    if profiles['batting'] is not None:
        batting_info = {k: v for k, v in profiles['batting'].items() if k != 'batter' and k != 'batter_id'}
        player_obj.set_batsman(**batting_info)
        
    if profiles['bowling'] is not None:
        bowling_info = {k: v for k, v in profiles['bowling'].items() if k != 'bowler' and k != 'bowler_id'}
        player_obj.set_bowler(**bowling_info)
    
    player_objects[player] = player_obj

# Print the player objects to check
for player_name, player_object in player_objects.items():
    print(player_object)
    print("\n---\n")



Player Profiles: {'JC Archer': {'batting': {'batter': 'JC Archer', 'batter_id': 427, 'Active Ratio_death': 0.7047619047619047, 'Striking Ratio_death': 0.2837837837837837, 'True SR_death': 7.38095238095238, 'True Avg_death': 4.956349206349207, 'Active Ratio_middle': 0.5, 'Striking Ratio_middle': 0.25, 'True SR_middle': 12.181818181818173, 'True Avg_middle': -11.095238095238097, 'Active Ratio': 0.0, 'Striking Ratio': 0.0, 'True SR_powerplay': -125.0, 'True Avg': -45.0}, 'bowling': {'bowler': 'JC Archer', 'bowler_id': 380, 'True Economy_death': 2.0633043265106004, 'True SR_death': -2.5162790697674406, 'dotpercentage_death': 0.2957317073170731, 'Containment_death': 1.5396825396825398, 'True Economy_middle': 2.2649557473776225, 'True SR_middle': 19.83333333333333, 'dotpercentage_middle': 0.4615384615384615, 'Containment_middle': 3.6666666666666665, 'True Economy_powerplay': 3.361418545676149, 'True SR_powerplay': 14.848416289592762, 'dotpercentage_powerplay': 0.6331360946745562, 'Containmen

In [66]:
class Player:
    def __init__(self, name, player_type):
        self.name = name
        self.player_type = player_type

class Batsman(Player):
    def __init__(self, name, batting_avg=None, batting_sr=None, active_ratio_death=None, striking_ratio_death=None, true_sr_death=None, true_avg_death=None,
                 active_ratio_middle=None, striking_ratio_middle=None, true_sr_middle=None, true_avg_middle=None,
                 active_ratio_powerplay=None, striking_ratio_powerplay=None, true_sr_powerplay=None, true_avg=None):
        super().__init__(name, "batsman")
        self.batting_avg = batting_avg
        self.batting_sr = batting_sr
        self.active_ratio_death = active_ratio_death
        self.striking_ratio_death = striking_ratio_death
        self.true_sr_death = true_sr_death
        self.true_avg_death = true_avg_death
        self.active_ratio_middle = active_ratio_middle
        self.striking_ratio_middle = striking_ratio_middle
        self.true_sr_middle = true_sr_middle
        self.true_avg_middle = true_avg_middle
        self.active_ratio_powerplay = active_ratio_powerplay
        self.striking_ratio_powerplay = striking_ratio_powerplay
        self.true_sr_powerplay = true_sr_powerplay
        self.true_avg = true_avg

class Bowler(Player):
    def __init__(self, name, bowling_economy=None, bowling_sr=None, true_economy_death=None, true_sr_death=None, dot_percentage_death=None, containment_death=None,
                 true_economy_middle=None, true_sr_middle=None, dot_percentage_middle=None, containment_middle=None,
                 true_economy_powerplay=None, true_sr_powerplay=None, dot_percentage_powerplay=None, containment_powerplay=None):
        super().__init__(name, "bowler")
        self.bowling_economy = bowling_economy
        self.bowling_sr = bowling_sr
        self.true_economy_death = true_economy_death
        self.true_sr_death = true_sr_death
        self.dot_percentage_death = dot_percentage_death
        self.containment_death = containment_death
        self.true_economy_middle = true_economy_middle
        self.true_sr_middle = true_sr_middle
        self.dot_percentage_middle = dot_percentage_middle
        self.containment_middle = containment_middle
        self.true_economy_powerplay = true_economy_powerplay
        self.true_sr_powerplay = true_sr_powerplay
        self.dot_percentage_powerplay = dot_percentage_powerplay
        self.containment_powerplay = containment_powerplay

def get_player_objects(all_players, dfbatting, dfbowling):
    player_objects = {}
    
    for player in all_players:
        batting_row = dfbatting[dfbatting['batter'] == player]
        bowling_row = dfbowling[dfbowling['bowler'] == player]
        
        if not batting_row.empty:
            # Extract only the required data for Batsman
            bat_data = batting_row.iloc[0]
            player_objects[player] = Batsman(
                name=player,
                batting_avg=bat_data.get('True Avg', None),
                batting_sr=bat_data.get('Striking Ratio', None),
                active_ratio_death=bat_data.get('Active Ratio_death', None),
                striking_ratio_death=bat_data.get('Striking Ratio_death', None),
                true_sr_death=bat_data.get('True SR_death', None),
                true_avg_death=bat_data.get('True Avg_death', None),
                # Continue for other batting attributes...
            )
        
        if not bowling_row.empty:
            # Extract only the required data for Bowler
            bowl_data = bowling_row.iloc[0]
            player_objects[player] = Bowler(
                name=player,
                bowling_economy=bowl_data.get('True Economy', None),
                bowling_sr=bowl_data.get('True SR', None),
                true_economy_death=bowl_data.get('True Economy_death', None),
                true_sr_death=bowl_data.get('True SR_death', None),
                dot_percentage_death=bowl_data.get('dotpercentage_death', None),
                containment_death=bowl_data.get('Containment_death', None),
                # Continue for other bowling attributes...
            )
            
    return player_objects

team1 = ["A Symonds", "AC Gilchrist","SR Watson","AD Russell","BA Stokes","CH Gayle","CH Morris","JC Archer","KA Pollard","KP Pietersen","Shakib Al Hasan"] 
team2 = ["SR Tendulkar", "Z Khan","RG Sharma","V Kohli","MS Dhoni","KH Pandya","HH Pandya","JJ Bumrah","SK Raina","YK Pathan","YS Chahal"]
player_profiles = get_player_objects( team2, dfbatting, dfbowling)
# Assuming dfbatting and dfbowling are pandas DataFrames and all_players is a list of player names
# player_objects = get_player_objects(all_players, dfbatting, dfbowling)


TypeError: __init__() got an unexpected keyword argument 'batter'

In [63]:
class Player:
    def __init__(self, name, role=None):
        self.name = name
        self.role = role
class Team:
    def __init__(self, name, players):
        self.name = name
        self.players = players
        self.score = 0
        self.wickets_lost = 0
        self.overs_bowled = 0
class Batsman(Player):
    def __init__(self, name, batting_avg=None, batting_sr=None, active_ratio_death=None, striking_ratio_death=None, true_sr_death=None, true_avg_death=None,
                 active_ratio_middle=None, striking_ratio_middle=None, true_sr_middle=None, true_avg_middle=None,
                 active_ratio_powerplay=None, striking_ratio_powerplay=None, true_sr_powerplay=None, true_avg=None):
        super().__init__(name, "batsman")
        self.batting_avg = batting_avg
        self.batting_sr = batting_sr
        self.active_ratio_death = active_ratio_death
        self.striking_ratio_death = striking_ratio_death
        self.true_sr_death = true_sr_death
        self.true_avg_death = true_avg_death
        self.active_ratio_middle = active_ratio_middle
        self.striking_ratio_middle = striking_ratio_middle
        self.true_sr_middle = true_sr_middle
        self.true_avg_middle = true_avg_middle
        self.active_ratio_powerplay = active_ratio_powerplay
        self.striking_ratio_powerplay = striking_ratio_powerplay
        self.true_sr_powerplay = true_sr_powerplay
        self.true_avg = true_avg

class Bowler(Player):
    def __init__(self, name, bowling_economy=None, bowling_sr=None, true_economy_death=None, true_sr_death=None, dot_percentage_death=None, containment_death=None,
                 true_economy_middle=None, true_sr_middle=None, dot_percentage_middle=None, containment_middle=None,
                 true_economy_powerplay=None, true_sr_powerplay=None, dot_percentage_powerplay=None, containment_powerplay=None):
        super().__init__(name, "bowler")
        self.bowling_economy = bowling_economy
        self.bowling_sr = bowling_sr
        self.true_economy_death = true_economy_death
        self.true_sr_death = true_sr_death
        self.dot_percentage_death = dot_percentage_death
        self.containment_death = containment_death
        self.true_economy_middle = true_economy_middle
        self.true_sr_middle = true_sr_middle
        self.dot_percentage_middle = dot_percentage_middle
        self.containment_middle = containment_middle
        self.true_economy_powerplay = true_economy_powerplay
        self.true_sr_powerplay = true_sr_powerplay
        self.dot_percentage_powerplay = dot_percentage_powerplay
        self.containment_powerplay = containment_powerplay

class Venue:
    def __init__(self, matches, total_run, dot_balls, is_wicket_delivery, delivery, fours, sixes, average_score, dot_ball_percentage, average_wickets_fallen, boundary_frequency):
        self.matches = matches
        self.total_run = total_run
        self.dot_balls = dot_balls
        self.is_wicket_delivery = is_wicket_delivery
        self.delivery = delivery
        self.fours = fours
        self.sixes = sixes
        self.average_score = average_score
        self.dot_ball_percentage = dot_ball_percentage
        self.average_wickets_fallen = average_wickets_fallen
        self.boundary_frequency = boundary_frequency
def get_player_data(df, player_name, role):
    player_df = df[df['player'] == player_name]  # Adjust 'player' to your column name for player

    if not player_df.empty:
        return player_df.iloc[0]

    print(f"Warning: {role} '{player_name}' not found in the dataset.")
    return None
class TeamPlayer:
    def __init__(self, name, dfbatting, dfbowling):
        self.name = name
        batting_data = get_player_data(dfbatting, name, "batsman")
        bowling_data = get_player_data(dfbowling, name, "bowler")

        self.batsman = self.create_batsman(name, batting_data) if batting_data is not None else None
        self.bowler = self.create_bowler(name, bowling_data) if bowling_data is not None else None

    def create_batsman(self, name, batting_data):
        # Example initialization of a Batsman object using batting_data
        # Adjust attribute assignments based on your Batsman class definition
        filtered_df = dfbowling[dfbowling['batter'] == name]
        if  not filtered_df.empty:
            player_data = filtered_df.iloc[0]
            return Batsman(
            name=name,
            player_data['True Avg'],
            batting_sr=player_data['True SR_powerplay'],
            active_ratio_death=player_data['Active Ratio_death'],
            striking_ratio_death=player_data['Striking Ratio_death'],
            true_sr_death=player_data['True SR_death'],
            true_avg_death=player_data['True Avg_death'],
            active_ratio_middle=player_data['Active Ratio_middle'],
            striking_ratio_middle=player_data['Striking Ratio_middle'],
            true_sr_middle=player_data['True SR_middle'],
            true_avg_middle=player_data['True Avg_middle'],
            active_ratio_powerplay=player_data['Active Ratio'],
            striking_ratio_powerplay=player_data['Striking Ratio'],
            true_sr_powerplay=player_data['True SR_powerplay'],
            true_avg=player_data['True Avg']
            # Include other attributes as necessary
        )
        else:
            return None

    def create_bowler(self, name, bowling_data):
        # Example initialization of a Bowler object using bowling_data
        # Adjust attribute assignments based on your Bowler class definition
        filtered_df = dfbowling[dfbowling['bowler'] == name]
        if  not filtered_df.empty:
            player_data = filtered_df.iloc[0]
            return Bowler(
            name=name,
            bowling_economy=player_data.get('bowling_economy', None),
            bowling_sr=player_data.get('bowling_sr', None),
            true_economy_death=player_data['True Economy_death'],
            true_sr_death=player_data['True SR_death'],
            dot_percentage_death=player_data['dotpercentage_death'],
            containment_death=player_data['Containment_death'],
            true_economy_middle=player_data['True Economy_middle'],
            true_sr_middle=player_data['True SR_middle'],
            dot_percentage_middle=player_data['dotpercentage_middle'],
            containment_middle=player_data['Containment_middle'],
            true_economy_powerplay=player_data['True Economy_powerplay'],
            true_sr_powerplay=player_data['True SR_powerplay'],
            dot_percentage_powerplay=player_data['dotpercentage_powerplay'],
            containment_powerplay=player_data['Containment_powerplay']
        )
        else:
            return None
    

# class TeamPlayer:
#     def __init__(self, name, dfbatting, dfbowling):
#         self.name = name
#         self.batsman = self.create_batsman(name, dfbatting)
#         self.bowler = self.create_bowler(name, dfbowling)
    
#     def create_batsman(self, name, dfbatting):
#         player_data_df = dfbowling[dfbowling['batter'] == name]
#         if not player_data_df.empty:
#             player_data = player_data_df.iloc[0]
#             # Continue with creating a Bowler object using player_data
#             return Batsman(
#                     name,
#                     player_data['True Avg'],
#                     batting_sr=player_data['True SR_powerplay'],
#                     active_ratio_death=player_data['Active Ratio_death'],
#                     striking_ratio_death=player_data['Striking Ratio_death'],
#                     true_sr_death=player_data['True SR_death'],
#                     true_avg_death=player_data['True Avg_death'],
#                     active_ratio_middle=player_data['Active Ratio_middle'],
#                     striking_ratio_middle=player_data['Striking Ratio_middle'],
#                     true_sr_middle=player_data['True SR_middle'],
#                     true_avg_middle=player_data['True Avg_middle'],
#                     active_ratio_powerplay=player_data['Active Ratio'],
#                     striking_ratio_powerplay=player_data['Striking Ratio'],
#                     true_sr_powerplay=player_data['True SR_powerplay'],
#                     true_avg=player_data['True Avg']
#                     # and so on for the rest of the batting attributes
#                     # Add other parameters as per the Batsman class definition
#                 )
#         else:
#             # Return a default Bowler object or None when no data is found
#             return None

        
#         # filtered_df = dfbatting[dfbatting['batter'] == name]
#         #     if not filtered_df.empty:
#         #         player_data = filtered_df.iloc[0]
#         # #player_data = dfbatting[dfbatting['batter'] == name].iloc[0]
#         # # Assuming you have a constructor for Batsman that takes these parameters
#         #         return Batsman(
#         #             name,
#         #             player_data['True Avg'],
#         #             batting_sr=player_data['True SR_powerplay'],
#         #             active_ratio_death=player_data['Active Ratio_death'],
#         #             striking_ratio_death=player_data['Striking Ratio_death'],
#         #             true_sr_death=player_data['True SR_death'],
#         #             true_avg_death=player_data['True Avg_death'],
#         #             active_ratio_middle=player_data['Active Ratio_middle'],
#         #             striking_ratio_middle=player_data['Striking Ratio_middle'],
#         #             true_sr_middle=player_data['True SR_middle'],
#         #             true_avg_middle=player_data['True Avg_middle'],
#         #             active_ratio_powerplay=player_data['Active Ratio'],
#         #             striking_ratio_powerplay=player_data['Striking Ratio'],
#         #             true_sr_powerplay=player_data['True SR_powerplay'],
#         #             true_avg=player_data['True Avg']
#         #             # and so on for the rest of the batting attributes
#         #             # Add other parameters as per the Batsman class definition
#         #         )
#         #     else:
#         #         return Batsmen(name)
        
    
    
#     # def create_bowler(self, name, dfbowling):
#     #     filtered_df = dfbowling[dfbowling['bowler'] == name]
#     #     if not filtered_df.empty:
#     #         player_data = filtered_df.iloc[0]
#     #             # Assuming you have a constructor for Bowler that takes these parameters
#     #         return Bowler(
#     #                 name,
#     #                 player_data.get('True Economy_powerplay', None),
#     #                 bowling_sr=player_data['True SR_powerplay'],
#     #                 true_economy_death=player_data['True Economy_death'],
#     #                 true_sr_death=player_data['True SR_death'],
#     #                 dot_percentage_death=player_data['dotpercentage_death'],
#     #                 containment_death=player_data['Containment_death'],
#     #                 true_economy_middle=player_data['True Economy_middle'],
#     #                 true_sr_middle=player_data['True SR_middle'],
#     #                 dot_percentage_middle=player_data['dotpercentage_middle'],
#     #                 containment_middle=player_data['Containment_middle'],
#     #                 true_economy_powerplay=player_data['True Economy_powerplay'],
#     #                 true_sr_powerplay=player_data['True SR_powerplay'],
#     #                 dot_percentage_powerplay=player_data['dotpercentage_powerplay'],
#     #                 containment_powerplay=player_data['Containment_powerplay']
#     #                 # Fill in other parameters as needed, using .get() with defaults of None
#     #             )
#     #     else:
#     #         return Bowler(name)
#     def create_bowler(self, name, dfbowling):
#     # Filter the dfbowling DataFrame for the specified name
#     player_data_df = dfbowling[dfbowling['bowler'] == name]
    
#     # Check if the filtered DataFrame is not empty
#     if not player_data_df.empty:
#         player_data = player_data_df.iloc[0]
#         # Continue with creating a Bowler object using player_data
#         return Bowler(
#             name=name,
#             bowling_economy=player_data.get('bowling_economy', None),
#             bowling_sr=player_data.get('bowling_sr', None),
#             true_economy_death=player_data['True Economy_death'],
#             true_sr_death=player_data['True SR_death'],
#             dot_percentage_death=player_data['dotpercentage_death'],
#             containment_death=player_data['Containment_death'],
#             true_economy_middle=player_data['True Economy_middle'],
#             true_sr_middle=player_data['True SR_middle'],
#             dot_percentage_middle=player_data['dotpercentage_middle'],
#             containment_middle=player_data['Containment_middle'],
#             true_economy_powerplay=player_data['True Economy_powerplay'],
#             true_sr_powerplay=player_data['True SR_powerplay'],
#             dot_percentage_powerplay=player_data['dotpercentage_powerplay'],
#             containment_powerplay=player_data['Containment_powerplay']
#             # Add other parameters as per the Bowler class definition
#             # Use .get() method with a default of None for each parameter
#         )
#     else:
#         # Return a default Bowler object or None when no data is found
#         return None

        

def create_venue_from_df(dfvenue, venue_identifier):
    # This is an example; adjust based on how you're identifying venues
    venue_data = dfvenue.loc[dfvenue['Venue_Name'] == venue_identifier].iloc[0]
    
    return Venue(
        matches=venue_data['Matches'],
        total_run=venue_data['total_run'],
        dot_balls=venue_data['Dot_Balls'],
        is_wicket_delivery=venue_data['isWicketDelivery'],
        delivery=venue_data['delivery'],
        fours=venue_data['Fours'],
        sixes=venue_data['Sixes'],
        average_score=venue_data['Average Score'],
        dot_ball_percentage=venue_data['Dot Ball Percentage'],
        average_wickets_fallen=venue_data['Average Wickets Fallen'],
        boundary_frequency=venue_data['Boundary Frequency']
    )



def Start(): 
    team1 = ["A Symonds", "AC Gilchrist","SR Watson","AD Russell","BA Stokes","CH Gayle","CH Morris","JC Archer","KA Pollard","KP Pietersen","Shakib Al Hasan"] 
    team2 = ["SR Tendulkar", "Z Khan","RG Sharma","V Kohli","MS Dhoni","KH Pandya","HH Pandya","JJ Bumrah","SK Raina","YK Pathan","YS Chahal"]    # Example names
    team1_players = [TeamPlayer(name, dfbatting, dfbowling) for name in team1]
    team2_players = [TeamPlayer(name, dfbatting, dfbowling) for name in team2]
    
   
    current_venue = create_venue_from_df(dfvenue, "Brabourne Stadium")
    # Setup Teams
    
    
    team1 = Team("Team 1", team1_players)
    team2 = Team("Team 2", team2_players)
    pitch = Pitch("balanced")
    
    # Simulate Innings
    print("First Innings:")
    simulate_innings(team1, team2, pitch)
    
    print("\nSecond Innings:")
    simulate_innings(team2, team1, pitch, 20)


def simulate_innings(batting_team, bowling_team, pitch, max_overs=20):
    current_batsman_index = 0
    non_striker_index = 1
    overs = 0
    total_score = 0
    wickets = 0

    while overs < max_overs and wickets < 10:
        bowler = bowling_team.players[overs % len(bowling_team.players)]  # Simple rotation, can be improved
        for ball in range(1, 7):  # Assuming no extras for simplicity
            if wickets >= 10:
                break
            batsman = batting_team.players[current_batsman_index]
            outcome = simulate_ball(batsman, bowler, pitch)
            outcome_to_runs = {"dot": 0, "single": 1, "double": 2, "triple": 3, "four": 4, "six": 6, "extra": 1, "wicket": 0}
            runs_scored = outcome_to_runs.get(outcome, 0)
            total_score += runs_scored

            if outcome == "wicket":
                wickets += 1
                current_batsman_index += 1
                if current_batsman_index == non_striker_index:  # Avoid having the same batsman and non-striker
                    current_batsman_index += 1

            print(f"{batting_team.name} | Over: {overs}.{ball} | {batsman.name} | {bowler.name} | Runs: {runs_scored} | Total: {total_score}/{wickets}")

            if ball == 6:  # Swap the batsmen at the end of the over
                current_batsman_index, non_striker_index = non_striker_index, current_batsman_index

        overs += 1

    batting_team.score = total_score
    batting_team.wickets_lost = wickets
    bowling_team.overs_bowled = overs
    print(f"Innings complete: {batting_team.name} {total_score}/{wickets} after {overs} overs")
def simulate_ball(batsman, bowler, pitch):
    # Adjust outcome probabilities based on player attributes and pitch condition
    base_probability = {
        "dot": 30,
        "single": 40,
        "double": 10,
        "triple": 5,
        "four": 10,
        "six": 4,
        "wicket": 1
    }

    # Adjust for batsman strength
    if batsman.batting_sr > 130:
        base_probability["four"] += 2
        base_probability["six"] += 2
        base_probability["dot"] -= 4

    # Adjust for bowler strength
    if bowler.bowling_economy < 7:
        base_probability["wicket"] += 2
        base_probability["dot"] += 3
        base_probability["four"] -= 2
        base_probability["six"] -= 2

    # Adjust for pitch condition
    if pitch.condition == "batsman_friendly":
        base_probability["four"] += 3
        base_probability["six"] += 2
    elif pitch.condition == "bowler_friendly":
        base_probability["dot"] += 5
        base_probability["wicket"] += 2

    # Determine the outcome of the ball
    outcome = random.choices(list(base_probability.keys()), weights=base_probability.values(), k=1)[0]
    return outcome





SyntaxError: positional argument follows keyword argument (2861251619.py, line 90)

In [61]:
Start()

IndexError: single positional indexer is out-of-bounds